In [54]:
import nibabel as nb
import numpy as np
from nilearn import plotting, image
import os
import metric_util as mt
import data_util as du
from scipy import stats
import import_util as im
from scipy import ndimage
from nilearn.masking import apply_mask
import carpet_plot

import sys
from matplotlib import pyplot as plt
from scipy.spatial import distance
from scipy.cluster import hierarchy
import pandas as pd

In [2]:
template_folder = "/work/scratch/tensor_completion/4D/noise/template"
gm_prob_file_name ="mni_icbm152_gm_tal_nlin_asym_09c.nii"
csf_prob_file_name= "mni_icbm152_wm_tal_nlin_asym_09c.nii"
wm_prob_file_name = "mni_icbm152_wm_tal_nlin_asym_09c.nii"
icbm_file_name = "mni_icbm152_t1_tal_nlin_asym_09c.nii"

gm_prob_path = os.path.join(template_folder, gm_prob_file_name)
csf_prob_path = os.path.join(template_folder, csf_prob_file_name)
wm_prob_path = os.path.join(template_folder, wm_prob_file_name)
icbm_file_path = os.path.join(template_folder, icbm_file_name)

mask_gm_path = os.path.join(template_folder, "gm_mask.nii")
subject_img_data_gm_path = "/work/scratch/tensor_completion/4D/noise/mask/subject_gm_mask.nii"

subject_scan_path = du.get_full_path_subject1()
subject_img = mt.read_image_abs_path(subject_scan_path)

gm_prob_img = mt.read_image_abs_path(gm_prob_path)
csf_prob_img = mt.read_image_abs_path(csf_prob_path)
wm_prob_img = mt.read_image_abs_path(wm_prob_path)
icmb_prob_img = mt.read_image_abs_path(icbm_file_path)

mask_gm = image.resample_to_img(gm_prob_img, subject_img, interpolation='nearest').get_data()
mask_wm = image.resample_to_img(wm_prob_img, subject_img, interpolation='nearest').get_data()
mask_csf = image.resample_to_img(csf_prob_img, subject_img, interpolation='nearest').get_data()
mask_icmb = image.resample_to_img(icmb_prob_img, subject_img, interpolation='nearest').get_data()
subject_img_data = np.array(subject_img.get_data())

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/ndimage/interpolation.py:472: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied for the matrix parameter has changed in scipy 0.18.0.
  "The behaviour of affine_transform with a one-dimensional "


In [3]:
tissue,dimsTs = im.import_nifti(icbm_file_path)

Unfolded 193x229x193 mask matrix using Fortran ordering


In [4]:
subject_img_data_gm = subject_img_data[mask_gm>0.5]

In [5]:
subject_img_data_gm.shape

(36346, 144)

In [6]:
subject_img_data_gm = subject_img_data_gm[subject_img_data_gm.std(axis=-1)!=0]

In [7]:
mask_gm[mask_gm <0.5] = 0.0

In [8]:
z_scored_data = np.nan_to_num(stats.zscore(subject_img_data, axis=-1))

In [9]:
def get_gm_mask(infile, treshold = 0.2):
 gm_img = mt.read_image_abs_path(infile)
 gm_data = np.array(gm_img.get_data())
 gm_mask = (gm_data > 0.2)
 gm_mask = ndimage.binary_closing(gm_mask, iterations=2)
 gm_mask_img = image.new_img_like(gm_img, gm_mask)
 return gm_mask_img

In [10]:
def get_wm_mask(infile, treshold = 0.2):
 wm_img = mt.read_image_abs_path(infile)
 wm_data = np.array(wm_img.get_data())
 wm_mask = (wm_data > 0.2)
 wm_mask = ndimage.binary_closing(wm_mask, iterations=2)
 wm_mask_img = image.new_img_like(wm_img, wm_mask)
 return wm_mask_img

In [11]:
gm_img = get_wm_mask(wm_prob_path)

In [12]:
def get_csf_mask(infile, treshold = 0.2):
 cf_img = mt.read_image_abs_path(infile)
 cf_data = np.array(cf_img.get_data())
 cf_mask = (cf_data > 0.2)
 cf_mask = ndimage.binary_closing(cf_mask, iterations=2)
 cf_mask_img = image.new_img_like(cf_img, cf_mask)
 return cf_mask_img

In [13]:
xx = get_csf_mask(csf_prob_path)

In [18]:

x_miss_path = "/work/scratch/tensor_completion/4D/noise/corrupted_subjects/artifacts/45/x_miss_45.nii"
x_miss_img = mt.read_image_abs_path(x_miss_path)

In [33]:
def resample_mask(src_img, target_img):
    resampled_img = image.resample_to_img(src_img, target_img, interpolation='continuous')
    data = resampled_img.get_data()
    data = (data > 0).astype('float32')
    return data

In [34]:
gm_mask = resample_mask(gm_img, subject_img)

In [35]:
subject_img_data = subject_img.get_data()

In [49]:
ntsteps = subject_img_data.shape[-1]
X_ts = subject_img_data.reshape(-1, ntsteps)

In [39]:
CF_ind = 1
GM_ind = 2
WM_ind = 3
ordering = []

In [42]:
def reorderMatrixRows(X,distanceMetric='euclidean',linkageMethod='average',doOptimalLeaf=False):
    "Get reordering of rows of a matrix by clustering"

    # Find (and exclude) constant voxels:
    isConstant = (np.std(X,axis=1,ddof=1)==0)
    if np.any(isConstant):
        X = X[~isConstant,:]
        print('%u constant voxels ignored' % np.sum(isConstant))

    # z-score the remaining voxels:
    X = stats.zscore(X,axis=1,ddof=1)

    print('Filtered to %u x %u time series' % (X.shape[0],X.shape[1]))

    # Compute condensed pairwise distance matrix:
    # DataFrame.corr(method='Pearson',min_periods=1)
    dij = distance.pdist(X,metric=distanceMetric)
    print('%u %s distances computed!' %(dij.shape[0],distanceMetric))

    # Check D is well-behaved:
    if not np.isfinite(dij).all():
        raise ValueError('Distance matrix contains non-finite values...')

    # Compute hierarchical linkage structure:
    Z = hierarchy.linkage(dij,method=linkageMethod,optimal_ordering=doOptimalLeaf)
    print('%u objects agglomerated using average linkage clustering!' %(X.shape[0]))

    # Get voxel ordering vector:
    if np.any(isConstant):
        # Extend to the full size
        nodeOrdering = np.zeros_like(isConstant,dtype=int)
        nodeOrdering[~isConstant] = hierarchy.leaves_list(Z)
    else:
        nodeOrdering = hierarchy.leaves_list(Z)
    return nodeOrdering

In [41]:
X_ts.shape

(153594, 144)

In [55]:
distanceMetric='euclidean'
linkageMethod='average'
doOptimalLeaf=False

In [52]:
X_ts_gm = subject_img_data[gm_mask==1]

In [ ]:
nodeOrdering = reorderMatrixRows(X_ts_gm,distanceMetric=distanceMetric,
                            linkageMethod=linkageMethod,doOptimalLeaf=doOptimalLeaf)

Filtered to 94314 x 144 time series


(53, 63, 46)